In [5]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get POSTGRES_ADDRESS from .env file
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")


In [6]:

# Create engine with the address
engine = create_engine(POSTGRES_ADDRESS)
def get_sim_runs():
    df = pd.read_sql_query("SELECT * FROM benchmark_runs_v2", engine)
    # Set pandas display options to make tables wider
    pd.set_option('display.max_columns', None)  # Show all columns
    pd.set_option('display.width', 1000)        # Set width to 1000 characters
    pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple lines
    # Drop the 'args' column if it exists in the DataFrame
    # if 'args' in df.columns:
    # df = df.drop(columns=['config_details_json'])
    return df.query("id>1")
sim_df = get_sim_runs()


In [19]:

def get_fp(row):
    if row["implementation"].startswith("nd"):
        ro_met =  row["metrics"]["nd_metrics"]
        row["fp_micro"] = ro_met["false_positive_count"]/ro_met["total_pairs"]
        row["fp_macro"] = ro_met["false_positive_rate"]
    else:
        ro_met =  row["metrics"]["cl_metrics"][-1]
        row["fp_micro"] =  ro_met["total_false_positive_count"]/ro_met["total_total_pairs"]
        row["fp_macro"] = ro_met["total_false_positive_rate"]
        
        

    return row
        
sim_df = sim_df.apply(get_fp, axis=1)
view_sim_df = sim_df.drop(columns=["ngram_size", "min_ngram_size", "num_perm", "limit_files", "num_nodes", "record_count", "duplicate_count", "input_file", "timestamp", "id"])

In [20]:
view_sim_df = view_sim_df.assign(exp_name=view_sim_df.output_dir.str.split("/").str[-1]).drop(columns=["output_dir", "implementation"])
view_sim_df

,notes,threshold,execution_time,total_size_gb,config_details_json,metrics,fp,fp_micro,fp_macro,exp_name
1,Experiment: datasize | Workflow: nd_cl | Limit...,0.7,1868.719052,2.972425,"{\n ""args"": {\n ""workflow"": ""nd_cl"",\n ...","{'nd_time': 1481.2580375671387, 'nd_metrics': ...",0.777375,0.777375,0.453621,nd_cl_datasize_files10_thresh0.7_perm256
2,Experiment: datasize | Workflow: cl_nd | Limit...,0.7,445.153823,2.972425,"{\n ""args"": {\n ""workflow"": ""cl_nd"",\n ...","{'cl_nd_time': 444.95437932014465, 'cl_metrics...",0.759419,0.759419,0.440791,cl_nd_datasize_files10_thresh0.7_perm256
3,Experiment: datasize | Workflow: nd_cl | Limit...,0.7,1892.470000,5.942862,"{\n ""args"": {\n ""workflow"": ""nd_cl"",\n ...","{'nd_time': 1439.3975429534912, 'nd_metrics': ...",0.793228,0.793228,0.467767,nd_cl_datasize_files20_thresh0.7_perm256
4,Experiment: datasize | Workflow: cl_nd | Limit...,0.7,649.734896,5.942862,"{\n ""args"": {\n ""workflow"": ""cl_nd"",\n ...","{'cl_nd_time': 649.5374999046326, 'cl_metrics'...",0.790332,0.790332,0.416310,cl_nd_datasize_files20_thresh0.7_perm256
5,Experiment: datasize | Workflow: nd_cl | Limit...,0.7,2253.195967,11.883736,"{\n ""args"": {\n ""workflow"": ""nd_cl"",\n ...","{'nd_time': 1634.2958981990814, 'nd_metrics': ...",0.822215,0.822215,0.470901,nd_cl_datasize_files40_thresh0.7_perm256
6,Experiment: datasize | Workflow: cl_nd | Limit...,0.7,1420.974992,11.883736,"{\n ""args"": {\n ""workflow"": ""cl_nd"",\n ...","{'cl_nd_time': 1420.7762591838837, 'cl_metrics...",0.819333,0.819333,0.430796,cl_nd_datasize_files40_thresh0.7_perm256
7,Experiment: threshold | Workflow: nd_cl | Limi...,0.6,4941.537140,11.883736,"{\n ""args"": {\n ""workflow"": ""nd_cl"",\n ...","{'nd_time': 4324.399260759354, 'nd_metrics': {...",0.731658,0.731658,0.361330,nd_cl_threshold_files40_thresh0.6_perm256
8,Experiment: threshold | Workflow: cl_nd | Limi...,0.6,4070.318676,11.883736,"{\n ""args"": {\n ""workflow"": ""cl_nd"",\n ...","{'cl_nd_time': 4070.123877763748, 'cl_metrics'...",0.725581,0.725581,0.347450,cl_nd_threshold_files40_thresh0.6_perm256
9,Experiment: threshold | Workflow: nd_cl | Limi...,0.7,2641.562159,11.883736,"{\n ""args"": {\n ""workflow"": ""nd_cl"",\n ...","{'nd_time': 1966.7177550792694, 'nd_metrics': ...",0.822215,0.822215,0.470901,nd_cl_threshold_files40_thresh0.7_perm256
10,Experiment: threshold | Workflow: cl_nd | Limi...,0.7,1649.138648,11.883736,"{\n ""args"": {\n ""workflow"": ""cl_nd"",\n ...","{'cl_nd_time': 1648.934634923935, 'cl_metrics'...",0.813762,0.813762,0.441867,cl_nd_threshold_files40_thresh0.7_perm256


In [22]:
view_sim_df.iloc[:7].to_csv("view_sim_df.csv", index=False)

In [17]:
view_sim_df.output_dir.tolist()

['/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/nd_cl_datasize_files10_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/cl_nd_datasize_files10_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/nd_cl_datasize_files20_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/cl_nd_datasize_files20_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/nd_cl_datasize_files40_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/cl_nd_datasize_files40_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/nd_cl_threshold_files40_thresh0.6_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/cl_nd_threshold_files40_thresh0.6_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/nd_cl_threshold_files40_thresh0.7_perm256',
 '/mnt/gcs_bucket/ray_experiment_outputs_20250414_135126/cl_nd_threshold_files40_thresh0.7_perm25

In [12]:
sim_df.metrics.iloc[0]

{'nd_time': 1481.2580375671387,
 'nd_metrics': {'duplicate_count': 76548,
  'execution_time': 58.258687257766724,
  'false_positive_rate': 0.45362053328706475,
  'false_positive_count': 4337294.0,
  'total_pairs': 5579412.0},
 'cl_time': 385.1235592365265,
 'cl_metrics': [{'inference_time': 23.118709325790405,
   'train_time': 134.10090136528015,
   'total_time': 161.22191190719604,
   'stage': 'stage1'},
  {'inference_time': 13.681020927429199,
   'train_time': 129.9850064277649,
   'total_time': 223.72447180747986,
   'stage': 'stage2',
   'total_false_positive_count': 0,
   'total_false_positive_rate': 0.0,
   'total_total_pairs': 0}]}

In [ ]:

# for a,b in zip(df.metrics.tolist(), df.implementation.tolist()):
#     fp = get_fp()
#     print(b,a)

In [20]:

for a,b in zip(df.metrics.tolist(), df.implementation.tolist()):

    fp = get_fp(a["cl_metrics"][-1])
    print(b,a)

ZeroDivisionError: division by zero

In [ ]:
for a,b in zip(df.metrics.tolist(), df.notes.tolist()):
    print(a,b)

In [ ]:

df.iloc[1:].to_csv("benchmark_runs.csv", index=False)

In [ ]:
df.execution_time.sum()/3600

In [ ]:
df.duplicate_count+df.record_count

In [ ]:
2.7*60

In [ ]:
752.379005/

In [ ]:
421_609

In [ ]:
13831082

In [ ]:
13_831_082

In [ ]:
200/

In [ ]:
1_749_756

In [ ]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("/home/ohadr/database_project_c/benchmark_results.db")

df = pd.read_sql_query("SELECT * FROM benchmark_runs", conn)
df

In [ ]:
print(df[["duplicate_count","record_count","execution_time", "total_size_gb"]].to_markdown())